### Video Surveillance - Anomaly event detection
<i> Other applications deep surveillance included theft identification, violence detection and detection of chance of explosion</i> 
Using spatial temporal Autoencoders to identify abnormal activities
The encoder part extracts the spatial and temporal Information, and then the decoder reconstruct the frames
The abnormal events are Identified by computing the reconstruction loss using Euclidean distance between original and reconstructed batch.
- Training a model for anomaly detection tasks
- The training video contains videos with normal situation (16 training video clips)
- The testing video contains videos with both standard and abnormal events (21 testing video clips)
- The videp contains 30652 frames in toatal

In [6]:
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import glob
import os
import cv2

from keras.layers import Conv3D, ConvLSTM2D, Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import imutils


## steps
- create an array holder
- create a path to store the converted video frames
- create a path to store the videos after converting them to images
- convert the images into array
- perform some image transformation/processing on the images that has been converted into an array

In [7]:
store_image = []
train_path = './train'
fps = 5
train_videos = os.listdir(train_path)
train_images_path = train_path+'/frames'
os.makedirs(train_images_path)

def store_inarray(image_path):
    image = load_img(image_path)
    image = img_to_array(image)
    image = cv2.resize(image, (227, 227), interpolation = cv2.INTER_AREA)
    gray = 0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

<h5> Extraxt frames/images from video and save </h5>
- for each of the video in the training video
- convert to images each video

In [8]:
for video in train_videos:
    eachVideo = cv2.VideoCapture(video)
    success, frame = eachVideo.read()
    if success:
        name = train_images_path + '.jpg'
        cv2.imwrite(name, frame)
    eachVideo.release()
    cv2.destroyAllWindows()
    images = os.listdir(train_images_path)
    for image in images:
        image_path = train_images_path + '/' + image
        store_inarray(image_path)

<h5> Store the converted images into numpy array of a separate file </h5>

In [9]:
store_image = np.array(store_image)
# a,b,c=store_image.shape
# store_image.resize(b,c,a)
# store_image = (store_image - store_image.mean()) / (store_image.std())
# store_image = np.clip(store_image, 0, 1)
# np.save('training.npy', store_image)

## Spatial temporal autoencoder architecture

In [11]:
stae_model = Sequential()

stae_model.add(Conv3D(filters=128, kernel_size=(11, 11, 1), strides=(4, 4, 1), padding='valid', input_shape=(227, 227, 10, 1), activation='tanh'))
stae_model.add(Conv3D(filters=64, kernel_size=(5, 5, 1), strides=(2, 2, 1), padding='valid', activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', dropout=0.4, recurrent_dropout=0.3, return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), strides=1, padding='same', dropout=0.3, return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), strides=1, return_sequences=True, padding='same', dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128, kernel_size=(5, 5, 1), strides=(2, 2, 1), padding='valid', activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1, kernel_size=(11, 11, 1), strides=(4, 4, 1), padding='valid', activation='tanh'))

stae_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

## Train the autoencoder with the converted numpy data

In [12]:
training_data = np.load('training.npy')
frames = training_data.shape[2]
frames = frames-frames%10

training_data = training_data[:,:,:frames]
training_data = training_data.reshape(-1, 227, 227, 10)
training_data = np.expand_dims(training_data, axis=4)
target_data = training_data.copy()

epochs =5 
batch_size = 1

callback_save = ModelCheckpoint("saved_model.h5", montor='mean_squared-error', save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

stae_model.fit(training_data, target_data, batch_size=batch_size, epochs=epochs, callbacks = [callbacksave, callback_early_stopping])
stae_model.save('saved_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'training.npy'